In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import imageio
import pandas as pd
# from data_prep_gasflow import features, targets, features_test, targets_test
from torchsummary import summary

In [2]:
# path = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/checkpoint_ohneneu_16dim.pth.tar'
path = 'checkp_xy_mit_16dim.pth.tar'

In [3]:
# map_location=torch.device('cpu')
checkpoint = torch.load(path,  map_location=lambda storage, loc: storage)
model_structure = print(checkpoint['model'])


Sequential(
  (0): Linear(in_features=4, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=64, bias=True)
  (11): ReLU()
  (12): Linear(in_features=64, out_features=64, bias=True)
  (13): ReLU()
  (14): Linear(in_features=64, out_features=32, bias=True)
  (15): ReLU()
  (16): Linear(in_features=32, out_features=16, bias=True)
  (17): ReLU()
  (18): Linear(in_features=16, out_features=16, bias=True)
)


In [4]:
# another way to define a network
net = torch.nn.Sequential(
        torch.nn.Linear(4, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 16),
        torch.nn.ReLU(),
        torch.nn.Linear(16, 16),
    )

In [5]:
model = net
model.load_state_dict(checkpoint['state_dict'])
model.eval()
summary(model, (2,4)) # 2 Dimensionen und 4 Inputs

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 2, 512]           2,560
              ReLU-2               [-1, 2, 512]               0
            Linear-3               [-1, 2, 512]         262,656
              ReLU-4               [-1, 2, 512]               0
            Linear-5               [-1, 2, 256]         131,328
              ReLU-6               [-1, 2, 256]               0
            Linear-7               [-1, 2, 256]          65,792
              ReLU-8               [-1, 2, 256]               0
            Linear-9               [-1, 2, 128]          32,896
             ReLU-10               [-1, 2, 128]               0
           Linear-11                [-1, 2, 64]           8,256
             ReLU-12                [-1, 2, 64]               0
           Linear-13                [-1, 2, 64]           4,160
             ReLU-14                [-1

In [6]:
# data_raw= pd.read_csv('C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/gasflowdata_complex_16dim.csv')
data_raw= pd.read_csv('gasflowdata_mitA_16dim.csv')

In [7]:
gas = data_raw
for ii in range(16):
    gas = gas.drop(['time_'+str(ii+1)], axis=1)
#     print(ii)
    
gas.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16,carbohydrates,proteins,lipids,dilution
0,1691.696473,0.194330,0.027472,0.020199,0.019492,0.019414,0.019405,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.814724,0.079125,0.106152,0.133840
1,949.997750,0.038927,0.013907,0.013027,0.012980,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.905792,0.090004,0.004204,0.179608
2,2507.140873,1.407618,0.086091,0.034676,0.027808,0.026578,0.026338,0.026290,0.026280,0.026278,0.026277,0.026277,0.026277,0.026277,0.026277,0.026277,0.126987,0.632277,0.240736,0.095542
3,1002.798536,0.045099,0.014782,0.013675,0.013611,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.913376,0.050319,0.036305,0.174718
4,1316.125423,0.090699,0.019329,0.016368,0.016138,0.016119,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.632359,0.198621,0.169020,0.152592


In [8]:
for field in ['dilution']:
# for field in ['dilution', 'Gasflow_end']:
    max_field = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field

In [9]:
gasflow_list = []
for ii in range(16):
    gasflow_list.append('Gasflow_' + str(ii+1))

print(gasflow_list)

['Gasflow_1', 'Gasflow_2', 'Gasflow_3', 'Gasflow_4', 'Gasflow_5', 'Gasflow_6', 'Gasflow_7', 'Gasflow_8', 'Gasflow_9', 'Gasflow_10', 'Gasflow_11', 'Gasflow_12', 'Gasflow_13', 'Gasflow_14', 'Gasflow_15', 'Gasflow_16']


In [10]:
for field in [gasflow_list]:
# for field in ['dilution', 'Gasflow_end']:
    max_field = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field

gas.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16,carbohydrates,proteins,lipids,dilution
0,0.645230,0.000069,0.000010,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.814724,0.079125,0.106152,0.669218
1,0.362338,0.000014,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.905792,0.090004,0.004204,0.898064
2,0.956248,0.000497,0.000030,0.000014,0.000011,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.126987,0.632277,0.240736,0.477721
3,0.382477,0.000016,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.913376,0.050319,0.036305,0.873614
4,0.501983,0.000032,0.000007,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.632359,0.198621,0.169020,0.762983


In [11]:
torch.manual_seed(0)  

In [12]:
sample = np.random.choice(gas.index, size=int(len(gas)*0.9), replace=False)
data_raw_1, test_data_1 = gas.loc[sample], gas.drop(sample)

# # split into features and targets
columns = gasflow_list

features, targets = data_raw_1.drop(columns, axis=1), data_raw_1[columns]
features_test, targets_test = test_data_1.drop(columns, axis=1), test_data_1[columns]

In [13]:
features_test.head()

,carbohydrates,proteins,lipids,dilution
26,0.743132,0.066746,0.190121,0.541366
36,0.694829,0.044385,0.260786,0.437854
38,0.950222,0.000639,0.049139,0.125446
50,0.276025,0.462288,0.261687,0.323354
58,0.585268,0.262664,0.152068,0.592892


In [14]:
targets_test.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16
26,0.858948,0.000248,0.000020,0.000011,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
36,0.999117,0.000963,0.000050,0.000019,0.000014,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
38,0.187498,0.480141,0.482258,0.536440,0.537352,0.537889,0.538194,0.538362,0.538455,0.538505,0.538523,0.538551,0.538559,0.538562,0.538562,0.538562
50,0.505059,0.008643,0.000255,0.000053,0.000024,0.000017,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015
58,0.767442,0.000137,0.000014,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009


In [15]:
# x_data = torch.tensor(features.values, dtype=torch.float)
# y_data = torch.tensor(targets.values, dtype=torch.float)

# x_test = torch.tensor(features_test.values, dtype=torch.float)
# y_test = torch.tensor(targets_test.values, dtype=torch.float)

x_data = checkpoint['x_data']
y_data = checkpoint['y_data']
x_test = checkpoint['x_test']
y_test = checkpoint['y_test']

In [16]:
def my_loss(output, target):
    loss = ((output - target)**2).sum()
    return loss

In [17]:
prediction_test = net(x_test)

In [18]:
print(features_test)

       carbohydrates  proteins    lipids  dilution
26          0.743132  0.066746  0.190121  0.541366
36          0.694829  0.044385  0.260786  0.437854
38          0.950222  0.000639  0.049139  0.125446
50          0.276025  0.462288  0.261687  0.323354
58          0.585268  0.262664  0.152068  0.592892
...              ...       ...       ...       ...
11943       0.053313  0.019364  0.927323  0.576024
11947       0.021425  0.103469  0.875106  0.793219
11963       0.789816  0.071269  0.138915  0.514665
11970       0.326048  0.189229  0.484724  0.669236
11985       0.173565  0.371696  0.454740  0.714555

[1200 rows x 4 columns]


## Find optimal dilution at specific substrate values

In [19]:
x_new_1 = x_test
print(x_new_1)

tensor([[0.8147, 0.0791, 0.1062, 0.1338],
        [0.1419, 0.5509, 0.3072, 0.1061],
        [0.6787, 0.1029, 0.2184, 0.1837],
        ...,
        [0.5061, 0.1651, 0.3288, 0.1336],
        [0.2483, 0.0245, 0.7271, 0.1883],
        [0.0395, 0.1701, 0.7904, 0.1899]])


In [20]:
# Erstelle Liste mit möglichen Dilutionen bei Substrateingang

dil_list = []
test_list = []
x_new = []
x_t = np.zeros((100,4))
# print(x_t)
for iii in range(100):
    dil_list.append((iii+1)/100)
    x_new = x_new_1[0].numpy()
    x_new[3] = (iii+1)/100
    test_list.append(x_new[:])
#     print(x_new)
    x_t[:][iii] = x_new 

In [21]:
print(x_t)

[[0.81472367 0.07912454 0.10615177 0.01      ]
 [0.81472367 0.07912454 0.10615177 0.02      ]
 [0.81472367 0.07912454 0.10615177 0.03      ]
 [0.81472367 0.07912454 0.10615177 0.04      ]
 [0.81472367 0.07912454 0.10615177 0.05      ]
 [0.81472367 0.07912454 0.10615177 0.06      ]
 [0.81472367 0.07912454 0.10615177 0.07      ]
 [0.81472367 0.07912454 0.10615177 0.08      ]
 [0.81472367 0.07912454 0.10615177 0.09      ]
 [0.81472367 0.07912454 0.10615177 0.1       ]
 [0.81472367 0.07912454 0.10615177 0.11      ]
 [0.81472367 0.07912454 0.10615177 0.12      ]
 [0.81472367 0.07912454 0.10615177 0.13      ]
 [0.81472367 0.07912454 0.10615177 0.14      ]
 [0.81472367 0.07912454 0.10615177 0.15000001]
 [0.81472367 0.07912454 0.10615177 0.16      ]
 [0.81472367 0.07912454 0.10615177 0.17      ]
 [0.81472367 0.07912454 0.10615177 0.18000001]
 [0.81472367 0.07912454 0.10615177 0.19      ]
 [0.81472367 0.07912454 0.10615177 0.2       ]
 [0.81472367 0.07912454 0.10615177 0.20999999]
 [0.81472367 

In [22]:
# Wandle Liste von Listen in Tensorwerte um
x_test_new = torch.tensor(x_t, dtype=torch.float)

In [23]:
# Nutze das Netz um Vorhersagen für die verschiedenen Dilutionen bei Substratwerten zu machen
pred_sample = net(x_test_new)
pred_sample = pred_sample.detach().numpy()
print(pred_sample)
pred_sample.shape

# schreibe die Ergebnisse der Methangasmenge in csv-file
import csv

with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(pred_sample)

[[ 0.12741563  0.2389076   0.22064358 ...  0.24384987  0.2439234
   0.24396598]
 [ 0.16466917  0.39794093  0.3983031  ...  0.44474193  0.44483882
   0.44500363]
 [ 0.21184413  0.5590171   0.5620807  ...  0.6268648   0.6268655
   0.6268593 ]
 ...
 [ 0.09571752 -0.33984125 -0.29107118 ... -0.2626774  -0.26106554
  -0.26057762]
 [ 0.09321794 -0.3441955  -0.29479504 ... -0.26603767 -0.2644052
  -0.26391107]
 [ 0.09071746 -0.34855092 -0.2985202  ... -0.2693991  -0.26774603
  -0.2672457 ]]


In [24]:
# bestimme den maximalen Gasstrom im Gleichgewicht

def max_value(pred_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
#     print(list_1)
    return max([sublist[-1] for sublist in pred_sample])

print(max_value(pred_sample))
max_sample = max_value(pred_sample)

# bestimme den zugehörigen Index

def index_finding(pred_sample, max_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
    indices = [i for i, x in enumerate(list_1) if x == max_sample]
    return indices

print(index_finding(pred_sample, max_sample))

0.9399083
[4]


In [25]:
# Gegenprobe

x_point = torch.tensor(x_t[:][4], dtype=torch.float)
pred_sample_point = net(x_point)
pred_sample_point = pred_sample_point.detach().numpy()
print(pred_sample_point)

[0.33338276 0.8587232  0.8482023  0.9408719  0.94131845 0.94100004
 0.94060916 0.9406493  0.9404236  0.9404025  0.9399234  0.9407346
 0.94055104 0.94060427 0.9402999  0.93990827]


In [26]:
carbo = 0.2
protein = 0.25
inert_x = 0.1
inert_s = 0.2
fat = 0.25

dilution = 0.01

In [30]:
model_ADM = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/Netze_Regressionen_ASIM/ADM_model/ADM_Sim.m'

In [28]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [31]:
eng.model_ADM(inert_s, inert_x, carbo, protein, fat, dilution)

MatlabExecutionError: Undefined function 'model_ADM' for input arguments of type 'double'.


In [9]:
%%file multi_lin_func.m

function p = multi_lin_func(a,b)
    % in-file comments can be added like this
    a = a+1;
    b = b+1;
    p = a+b;
end

Overwriting multi_lin_func.m


In [10]:
multi_lin_func(1,2)

NameError: name 'multi_lin_func' is not defined

# ab hier normaler Code

In [32]:
# Testing a sample of testdata
pred_sample = net(x_test[0])
pred_sample = pred_sample.detach().numpy()
y_real = y_test[0].detach().numpy()

print(x_test[0])
print(pred_sample)
print(y_real)


# showing deviation
print(y_real - pred_sample)


time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]
t = time_series.loc[0].to_numpy()

plt.scatter(t, y_real)
plt.scatter(t, pred_sample)
plt.show()

tensor([0.8147, 0.0791, 0.1062, 1.0000])
[ 0.09071743 -0.34855103 -0.29852033 -0.30029356 -0.28475624 -0.2765141
 -0.27353525 -0.26895458 -0.2696733  -0.274648   -0.27113736 -0.26793072
 -0.27137    -0.26939923 -0.2677461  -0.26724577]
[6.4522964e-01 6.8613990e-05 9.6559243e-06 7.8648100e-06 7.5854050e-06
 7.5538346e-06 7.5499147e-06 7.5493431e-06 7.5492367e-06 7.5492103e-06
 7.5492048e-06 7.5491998e-06 7.5491994e-06 7.5491994e-06 7.5491994e-06
 7.5491994e-06]
[0.5545122  0.34861964 0.29852998 0.30030143 0.28476384 0.27652165
 0.2735428  0.26896212 0.26968083 0.27465555 0.2711449  0.26793826
 0.27137753 0.26940677 0.26775363 0.2672533 ]


In [33]:
loss = my_loss(prediction_test, y_test)
print(loss)

tensor(85.5266, grad_fn=<SumBackward0>)


In [34]:
score_model_pre = 100-(abs((prediction_test - y_test)/y_test))
score_model = torch.mean(score_model_pre)
print(score_model)

prediction_train = net(x_data)
score_model_pre = 100-(abs((prediction_train - y_data)/y_data))
score_model = torch.mean(score_model_pre)
print(score_model)

print(prediction_test.detach().numpy())
print(y_test.detach().numpy())

abw = ((prediction_test.detach().numpy() - y_test.detach().numpy())**2)
print(abw.sum())
print(abw.shape)

max_list = []
for ii in range(16):
    max_list.append(max(abw[ii,:]))
print(max_list)

tensor(-45.1772, grad_fn=<MeanBackward0>)
tensor(96.6993, grad_fn=<MeanBackward0>)
[[ 6.4616984e-01 -4.7266483e-05  1.5363097e-05 ...  1.8894672e-05
   1.8209219e-05  1.7240644e-05]
 [ 8.7011850e-01  4.1598082e-04  3.4973025e-05 ...  1.0550022e-05
   1.0460615e-05  1.0371208e-05]
 [ 3.4827200e-01  1.2359023e-04  2.6479363e-05 ...  1.7613173e-05
   1.7985702e-05  1.8715858e-05]
 ...
 [ 6.4195526e-01  1.5226007e-04  2.2202730e-05 ...  2.3275614e-05
   2.2545457e-05  2.1561980e-05]
 [ 3.3384410e-01  2.1347404e-04  2.9742718e-05 ...  1.5795231e-05
   1.6406178e-05  1.7508864e-05]
 [ 3.2752940e-01  1.5044212e-04 -1.0073185e-05 ... -2.1666288e-05
  -2.0742416e-05 -1.9431114e-05]]
[[6.45229638e-01 6.86139902e-05 9.65592426e-06 ... 7.54919938e-06
  7.54919938e-06 7.54919938e-06]
 [8.67476821e-01 2.62590096e-04 1.99666647e-05 ... 9.37213827e-06
  9.37213827e-06 9.37213827e-06]
 [3.47151935e-01 1.24998696e-05 4.75467732e-06 ... 4.95788299e-06
  4.95788299e-06 4.95788299e-06]
 ...
 [6.45257831e-0

In [35]:
import numpy as np
import numpy.matlib
# import mplcursors

time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]


t = time_series.loc[0].to_numpy()
time_series.head()

,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,time_10,time_11,time_12,time_13,time_14,time_15,time_16
0,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
1,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
2,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
3,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
4,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300


In [36]:
%matplotlib qt

In [37]:


from mpl_toolkits.mplot3d import Axes3D
y1 = y_test.numpy()
y2 = prediction_test.detach().numpy()
t = time_series.loc[0].to_numpy()
# t = np.matlib.repmat(t, len(y1), 1)
print(y1[0,3])
print(y2[0,3])

for i in range(70,80):
    fig = plt.figure()
    plt.plot(t, y1[i+10,:], 'bo--', label=r'$\dot m(CH_4)$ ADM1' , alpha=0.5)
    plt.plot(t, y2[i+10,:], marker=6, color='r', label=r'$\dot m(CH_4)$ DNN', alpha=0.5)
    plt.legend(loc='best')
    plt.xlabel('Zeit [d]')
    plt.ylabel(r'$\dot m(CH_4)(skaliert)$')
    plt.show()

7.86481e-06
2.0116568e-05


In [69]:
# path = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/checkpoint_ohneAuswasch_16dim_1.pth.tar'
# checkpoint = {'x_test': x_test, 'y_test': y_test, 'x_data': x_data, 'y_data': y_data,
#               'model': model,
#               'state_dict': model.state_dict()}

# torch.save(checkpoint, path)

In [70]:
# checkpoint_1 = torch.load(path,  map_location=lambda storage, loc: storage)

In [71]:
# x_data_1 = checkpoint['x_data'] 

In [73]:
# print(x_data_1, x_data)

tensor([[0.4294, 0.0866, 0.4840, 1.0000],
        [0.4852, 0.3456, 0.1692, 0.9925],
        [0.4716, 0.4852, 0.0433, 0.5376],
        ...,
        [0.6208, 0.3367, 0.0425, 0.4188],
        [0.3414, 0.5566, 0.1020, 0.5719],
        [0.6203, 0.2340, 0.1457, 0.9593]]) tensor([[0.4294, 0.0866, 0.4840, 1.0000],
        [0.4852, 0.3456, 0.1692, 0.9925],
        [0.4716, 0.4852, 0.0433, 0.5376],
        ...,
        [0.6208, 0.3367, 0.0425, 0.4188],
        [0.3414, 0.5566, 0.1020, 0.5719],
        [0.6203, 0.2340, 0.1457, 0.9593]])
